In [1]:
import numpy as np
import pandas as pd
import codecs
import sys
import re
import json
import requests
import os
import time
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

In [2]:
f = codecs.open(u'КоАП.txt', encoding = 'utf-8')
text = f.read()
part1, part2 = text.split(u'Раздел II. ОСОБЕННАЯ ЧАСТЬ')
f.close()

In [70]:
class Part:
    
    def __init__(self, text, num):
        self._number = re.sub(r'\n| ', '', num)
        if (text):
            text = re.split(r'-\nвлечет|-\nвлекут', text)
            self._text = text[0]
            if (len(text)>1):
                self._answer = text[1]
            else:
                self._answer = None
        else: 
            self._text, self._answer = None, None
    def genAnswer(self):
        pass
    
    def __str__ (self):
        return (str(self._number) + " -> " + self._text + "->" + self._answer)


class Article:
    
    def __init__(self, text):
        number_of_article_re = re.compile('\d+\.\d+\.\d+-\d+\. |\d+\.\d+\.\d+\. |\d+\.\d+\. ')
        number_of_article = number_of_article_re.match(text).group(0)
        text = re.sub(number_of_article, '', text)

        self._title = re.match(r'.+', text).group(0)
        text = re.sub(r'^.+', '', text)
        self._title = self._title[:-1]
        
        number_of_article = re.split(r'\.', number_of_article)       
        number_of_article = number_of_article[:-1]
        
        self._chapter = number_of_article[0]
        self._number = number_of_article[1]
        if (len(number_of_article)>2):
            self._subnumber = number_of_article[2]
        else:
            self._subnumber = None
        
        text = re.sub(r'\(.+Федерал.+\)', '', text)
        text = re.sub(r'\s\s+', '\n', text)
        text = re.sub(r'^\n', '', text)
        self._setParts(text)
        
    def _setParts(self, text):
        self._parts = []
        sep = re.findall(r'^\d+. |^\d+.\d+. |\n\d+. |\n\d+.\d+. ' , text)
        parts = re.split(r'^\d+. |^\d+.\d+. |\n\d+. |\n\d+.\d+. ' , text)
        if (len(parts) > 1):
            for i, part in enumerate (parts[1:]):
                self._parts.append(Part(part, sep[i]))
        else:
            self._parts.append(Part(parts[0], '0'))
        
    
    def __str__ (self):
        return (self._parts)
    
    def genText(self):
        pass

In [71]:
doc_list = []
list_of_articles = part2.split('Статья ')

for article in list_of_articles:
    if (article):
        doc_list.append(Article(article))

In [72]:
data = []

for doc in doc_list:
    for part in doc._parts:
        data.append([doc._chapter, doc._number, doc._subnumber, doc._title, part._number, part._text, part._answer])
        
datapd = pd.DataFrame(data, columns = ['chapter', 'number', 'subnumber', 'title', 'part', 'text', 'answer'])
datapd.to_csv("Dump.csv")

In [6]:
def get_words(text):
    tokenizer = RegexpTokenizer("\w[^!.;,?\-\s]*\w|\w")
    s = []
    s = tokenizer.tokenize(text)
    return s

In [7]:
#for getting stopwords
#nltk.download

def get_tokens(words):
    stop_words = stopwords.words("russian")
    morph = MorphAnalyzer()
    tokens = []
    for i in words:
        if (i not in stop_words):
            tokens.append(morph.parse(i)[0].normal_form)
    return tokens

In [8]:
with codecs.open("аббревиатуры.txt") as f:
    abbrevations = f.read()
    
abbrevations = re.split(r'\n', abbrevations)
abbrevations = dict([(re.split(',', e)[0], re.split(',', e)[1]) for e in abbrevations])

In [9]:
with codecs.open("changes.txt") as f:
    changes = f.read()
    
changes = re.split(r'\n', changes)
changes = dict([(re.split(',', e)[0], re.split(',', e)[1]) for e in changes])

In [10]:
queries = pd.read_excel('Запросы.xlsx')

In [11]:
queries = queries['Запрос'].tolist()

In [12]:
queries_data = pd.DataFrame(queries, columns=['query'])

In [13]:
def good_search(query):
    query = query.lower()
    query = get_words(query)
    
    abb = [e for e in query if (e in abbrevations)]
    query = [e for e in query if not (e in abbrevations)]
    for i in abb:
        query += get_words(abbrevations[i])
    query = get_tokens(query)
    
    ch = [e for e in query if (e in changes)]
    query = [e for e in query if not (e in changes)]
    
    for i in ch:
        query += get_tokens(get_words(changes[i]))
    return query


In [206]:
def equality(query, text_list):
    words_count = len(query) - 1
    words = set(query)
    max_count = 0
    max_index = -1
    for num, row in enumerate (text_list.text):
        if (row):
            text = set(good_search(row))
            count = len(text.intersection(words))
            if count > max_count:
                max_count, max_index = count, num
    if max_index > -1:
        data_index = text_list.index[max_index]
    else:
        data_index = -1
    if words_count == 0:
        words_count = 1000
    return data_index, max_count/words_count

In [15]:
def query_process(queries_list):
    final = []
    for query in queries_list:
        if query not in processed_queries:
            time.sleep(1)
            s = '%20'
            goodquery = good_search(query)
            chquery = s.join(goodquery)
            url = 'https://yandex.ru/search/?text=' + chquery + '&lr=213&site=www.consultant.ru%2Fdocument%2Fcons_doc_LAW_34661%2F'
            r = requests.get(url)
            soup = BeautifulSoup(r.text, "lxml")
            punct = soup.find_all('li')
            punct = [e for e in punct if not e.has_attr('aria-label')]
            if (len (punct) > 0):
                txt = "".join(punct[0].strings)
                print (txt)
                txt = re.search(r'Статья (\d+\.\d+\.\d+-\d+\. |\d+\.\d+\.\d+\. |\d+\.\d+\. )', txt)
                if (txt is not None):
                    txt = txt.group(0)
                    num = re.sub(r"Статья ", "", txt)

                    num = re.split(r'\.', num)       
                    num = num[:-1]
                    if (len(num)>2):
                        text_list = datapd[(datapd.chapter == num[0]) & 
                               (datapd.number == num[1]) & 
                               (datapd.subnumber == num[2])]
                    else:
                        text_list = datapd[(datapd.chapter == num[0]) & 
                               (datapd.number == num[1]) & 
                               (datapd.subnumber.isnull())]
                    num, prec = equality(goodquery, text_list)
                    if num == -1:
                        num = None
                        prec = None
                    print (query)
                    req = {
                        'query': query,
                        'id': str(num),
                        'precision': str(prec)
                    }
                    final.append(req)
                    f.write(json.dumps(final[-1]) + '\n')
    f.close()
    return final

In [155]:
#main cell - temp.txt for saving results
query_records = []
tmp_file_name = 'temp.txt'
if os.path.exists(tmp_file_name):
    with open(tmp_file_name) as f:
        for line in f:
            try:
                query_records.append(json.loads(line))
            except:
                continue
print (len(query_records))
processed_queries = set()
for r in query_records:
    processed_queries.add(r['query'])
f = open(tmp_file_name, 'a')
#-----------------change values here-----|
#---------------------------------------\/
query_records += query_process(queries[0:300])
df_records = pd.DataFrame(query_records, columns=['query', 'id', 'precision'])
df_full = pd.merge(queries_data, df_records, on="query", how="left")

247
1Тютрюмов И.М. Законы гражданские с разъяснениями...civil.consultant.ru›Электронная библиотека›КонсультантПлюс›37/page…Сохранённая копияПожаловатьсяПропуском 10-летней давности казна теряет право обращения взыскания на ... 208. С неисправного подрядчика взыскивается: 1) штраф, согласно правилам, в ст. 87-90...
1Тютрюмов И.М. Законы гражданские с разъяснениями...civil.consultant.ru›Электронная библиотека›КонсультантПлюс›36/page…Сохранённая копияПожаловаться...из нее багаж без расплаты определяются штраф или тюрьма. ... кредитных билетов того выпуска на другие с назначением срока для представления к обмену, то...
1КоАП РФ, Глава 15. АДМИНИСТРАТИВНЫЕ...Consultant.ru›document/cons_doc_LAW_34661/…/Пожаловаться...поручения о перечислении налога (сбора), страхового взноса, пеней, штрафа. ... Деятельность по привлечению денежных средств и (или) иного имущества.
1Тютрюмов И.М. Законы гражданские с разъяснениями...civil.consultant.ru›КонсультантПлюс›Книга четвертаяСохранённая копияПожаловать

KeyboardInterrupt: 

In [157]:
df_full.to_csv("fromYand.csv")

In [18]:
query_process(['штраф за парковку на газоне'])

[]

In [73]:
datapd

,chapter,number,subnumber,title,part,text,answer
0,5,1,None,Нарушение права гражданина на ознакомление со ...,0,Нарушение права гражданина на ознакомление со ...,наложение административного штрафа в размере ...
1,5,2,None,Утратила силу. - Федеральный закон от 04.07.20...,0,None,None
2,5,3,None,"Неисполнение решения избирательной комиссии, к...",1.,"Неисполнение решения избирательной комиссии, к...",наложение административного штрафа на должнос...
3,5,3,None,"Неисполнение решения избирательной комиссии, к...",2.,"Непредставление государственными органами, орг...",наложение административного штрафа на должнос...
4,5,4,None,Нарушение порядка представления сведений об из...,0,Нарушение установленного законом порядка предс...,наложение административного штрафа на должнос...
5,5,5,None,Нарушение порядка участия средств массовой инф...,1.,"Нарушение главным редактором, редакцией средст...",наложение административного штрафа на граждан...
6,5,5,None,Нарушение порядка участия средств массовой инф...,2.,Непредоставление государственной или муниципал...,наложение административного штрафа на должнос...
7,5,6,None,"Нарушение прав члена избирательной комиссии, к...",1.,"Нарушение прав члена избирательной комиссии, к...",наложение административного штрафа на граждан...
8,5,6,None,"Нарушение прав члена избирательной комиссии, к...",2.,"Выдача председателем, заместителем председател...",наложение административного штрафа в размере ...
9,5,7,None,Отказ в предоставлении отпуска для участия в в...,0,Отказ работодателя предоставить предусмотренны...,наложение административного штрафа в размере ...


In [301]:
def word_to_num(text):
    dict_num = {
        'полтора': 1.5,
        'один': 1,
        'два': 2,
        'три': 3,
        'четыре': 4,
        'пять': 5,
        'шесть': 6,
        'семь': 7,
        'восемь': 8,
        'девять': 9,
        'десять': 10,
        'одиннадцать': 11,
        'двенадцать': 12, 
        'тринадцать': 13,
        'четырнадцать': 14, 
        'пятнадцать': 15,
        'шестнадцать': 16, 
        'семнадцать': 17,
        'восемнадцать': 18, 
        'девятнадцать': 19,
        'двадцать': 20, 
        'тридцать': 30,
        'сорок': 40, 
        'пятьдесят': 50,
        'шестьдесят': 60, 
        'семьдесят': 70,
        'восемьдесят': 80, 
        'девяносто': 90,
        'сто': 100, 
        'двести': 200,
        'триста': 300, 
        'четыреста': 400,
        'пятьсот': 500, 
        'шестьсот': 600,
        'семьсот': 700,
        'восемьсот': 800,
        'девятьсот': 900
    }
    dict_mult = {
        'тысяча': 1000,
        'миллион': 1000000,
        'миллиард': 1000000000
    }
    lst = get_tokens(get_words(text))
    result = 0
    for num in lst:
        if num in dict_num:
            result += dict_num[num]
        if num in dict_mult:
            result *= dict_mult[num]
    return result

print (word_to_num('одного миллиона'))

1000000


In [305]:
def gen_answer(text):
    result = '-> '
    if text:
        text = re.split(r' или ', text)
        for item in text:
            if re.search(r'предупреждение', item):
                result += 'предупреждение или '
            elif re.search(r'штраф', item):
                result += 'штраф '
                fine = re.split(r';', item)
                for part in fine:
                    if re.search(r'рублей', part):
                        if re.search(r'должностных', part):
                            result += 'для должн. лиц - '
                        elif re.search(r'юридических', part):
                            result += 'для юр. лиц - '
                        if re.search(r' от .* до .* рублей', part):
                            parts = re.split(r' от | до | рублей', part)
                            result += 'от ' + str(word_to_num(parts[1])) + ' до ' + str(word_to_num(parts[2])) + ' рублей; '
                        elif re.search(r'в размере', part):
                            parts = re.split(r' в размере | рублей', part)
                            result += 'в размере ' + str(word_to_num(parts[1])) + ' рублей; '
                        else:
                            parts = re.split(r' - | рублей', part)
                            result += 'в размере ' + str(word_to_num(parts[1])) + ' рублей; '
        
        
        '''text = re.search(r'штрафа.*рублей', text)
        if text is not None:
            result += '-> '
            text = re.split(r';', text.group(0))
            for t in text:
                if (re.search(r' от ', t)):
                    parts = re.split(r' от | до | рублей', t)
                    if re.search(r'должностных', parts[0]):
                        result += 'для должн. лиц - '
                    elif re.search(r'юридических', parts[0]):
                        result += 'для юр. лиц - '
                    result += 'от ' + str(word_to_num(parts[1])) + ' до ' + str(word_to_num(parts[2])) + ' рублей; '
                else:
                    parts = re.split(r' в размере | рублей | - ', t)
                    if re.search(r'должностных', parts[0]):
                        result += 'для должн. лиц - '
                    elif re.search(r'юридических', parts[0]):
                        result += 'для юр. лиц - '
                    result += str(word_to_num(parts[1])) + ' рублей; '
        else:
            result = None'''
    else:
        result = None
    if result == '-> ' or result == '-> штраф ':
        result = None
    return result
    
gen_answer('наложение административного штрафа в случае, если определена кадастровая стоимость земельного участка, на граждан в размере от 0,5 до 1 процента кадастровой стоимости земельного участка, но не менее десяти тысяч рублей; на должностных лиц - от 1 до 1,5 процента кадастровой стоимости земельного участка, но не менее двадцати тысяч рублей; на юридических лиц - от 1,5 до 2 процентов кадастровой стоимости земельного участка, но не менее ста тысяч рублей, а в случае, если не определена кадастровая стоимость земельного участка, на граждан в размере от десяти тысяч до двадцати тысяч рублей; на должностных лиц - от двадцати тысяч до пятидесяти тысяч рублей; на юридических лиц - от ста тысяч до двухсот тысяч рублей.')

'-> штраф от 0 до 10000 рублей; для должн. лиц - от 0 до 20000 рублей; для юр. лиц - от 0 до 100000 рублей; для должн. лиц - от 20000 до 50000 рублей; для юр. лиц - от 100000 до 200000 рублей; '

In [115]:
datapd['answer_out'] = None
datapd['text_out'] = None

In [283]:
def short_text(text):
    text = re.split(r'\s', text)
    out_text = text[0]
    del (text[0])
    while len (out_text) < 70 and len (text) > 0:
        out_text += ' ' + text[0]
        del (text[0])
    #if len(re.split(r' ', text[0])) > 6:
    #    s = ' '
    #    text = re.split(r' ', text[0])
    #    text = s.join(text[:4])
    #else:
    #    text = text[0]
    if len (text) == 0:
        return (out_text + '.')
    else: 
        return (out_text + '...')
print (short_text('Непредоставление перевозчиком пассажирам в случаях и в порядке, которые установлены федеральным законом, информации о страховщике и о договоре обязательного страхования гражданской ответственности перевозчика за причинение вреда жизни, здоровью, имуществу пассажиров'))

Непредоставление перевозчиком пассажирам в случаях и в порядке, которые...


In [287]:
datapd

,chapter,number,subnumber,title,part,text,answer,answer_out,text_out
0,5,1,None,Нарушение права гражданина на ознакомление со ...,0,Нарушение права гражданина на ознакомление со ...,наложение административного штрафа в размере ...,-> от 1000 до 1500 рублей;,ст 5.1 КоАП РФ. Нарушение права гражданина на ...
1,5,2,None,Утратила силу. - Федеральный закон от 04.07.20...,0,None,None,None,ст 5.2 КоАП РФ. Утратила силу. - Федеральный з...
2,5,3,None,"Неисполнение решения избирательной комиссии, к...",1.,"Неисполнение решения избирательной комиссии, к...",наложение административного штрафа на должнос...,-> для должн. лиц - от 1000 до 2000 рублей; дл...,ч.1. ст 5.3 КоАП РФ. Неисполнение решения изби...
3,5,3,None,"Неисполнение решения избирательной комиссии, к...",2.,"Непредставление государственными органами, орг...",наложение административного штрафа на должнос...,-> для должн. лиц - от 1000 до 1500 рублей; дл...,ч.2. ст 5.3 КоАП РФ. Непредставление государст...
4,5,4,None,Нарушение порядка представления сведений об из...,0,Нарушение установленного законом порядка предс...,наложение административного штрафа на должнос...,-> для должн. лиц - от 1000 до 5000 рублей;,ст 5.4 КоАП РФ. Нарушение порядка представлени...
5,5,5,None,Нарушение порядка участия средств массовой инф...,1.,"Нарушение главным редактором, редакцией средст...",наложение административного штрафа на граждан...,-> от 500 до 2500 рублей; для юр. лиц - от 300...,ч.1. ст 5.5 КоАП РФ. Нарушение главным редакто...
6,5,5,None,Нарушение порядка участия средств массовой инф...,2.,Непредоставление государственной или муниципал...,наложение административного штрафа на должнос...,-> для должн. лиц - от 3000 до 4000 рублей; дл...,ч.2. ст 5.5 КоАП РФ. Непредоставление государс...
7,5,6,None,"Нарушение прав члена избирательной комиссии, к...",1.,"Нарушение прав члена избирательной комиссии, к...",наложение административного штрафа на граждан...,-> от 500 до 1000 рублей; для должн. лиц - от ...,ч.1. ст 5.6 КоАП РФ. Нарушение прав члена изби...
8,5,6,None,"Нарушение прав члена избирательной комиссии, к...",2.,"Выдача председателем, заместителем председател...",наложение административного штрафа в размере ...,-> от 1500 до 2000 рублей;,"ч.2. ст 5.6 КоАП РФ. Выдача председателем, зам..."
9,5,7,None,Отказ в предоставлении отпуска для участия в в...,0,Отказ работодателя предоставить предусмотренны...,наложение административного штрафа в размере ...,-> от 1500 до 2000 рублей;,ст 5.7 КоАП РФ. Отказ в предоставлении отпуска...


In [306]:
for index, row in datapd.iterrows():
    row['answer_out'] = gen_answer(row['answer'])
    #short = ''
    #if row['part'] != '0' and row['part'] != 0:
    #    short += 'ч.' + row['part'] + ' '
    #short += 'ст ' + row['chapter'] + '.' + row['number']
    #if row['subnumber']:
    #    short += '.' + row['subnumber']
    #short += ' КоАП РФ. ' 
    #if row['part'] == '0' or row['part'] == 0:
    #    short += short_text(row['title'])
    #else:
    #    short += short_text(row['text'])
    #row['text_out'] = short
    print (row['answer_out'])

-> штраф от 1000 до 1500 рублей; 
None
-> штраф для должн. лиц - от 1000 до 2000 рублей; для юр. лиц - от 10000 до 20000 рублей; 
-> штраф для должн. лиц - от 1000 до 1500 рублей; для юр. лиц - от 10000 до 15000 рублей; 
-> штраф для должн. лиц - от 1000 до 5000 рублей; 
-> штраф для должн. лиц - от 500 до 2500 рублей; для юр. лиц - от 30000 до 100000 рублей; 
-> штраф для должн. лиц - от 3000 до 4000 рублей; для юр. лиц - от 20000 до 30000 рублей; 
-> штраф от 500 до 1000 рублей; для должн. лиц - от 1000 до 2000 рублей; 
-> штраф от 1500 до 2000 рублей; 
-> штраф от 1500 до 2000 рублей; 
-> штраф от 1000 до 2500 рублей; для должн. лиц - от 2000 до 5000 рублей; для юр. лиц - от 20000 до 100000 рублей; 
-> штраф от 1000 до 1500 рублей; для должн. лиц - от 2000 до 3000 рублей; для юр. лиц - от 20000 до 30000 рублей; 
-> штраф от 1000 до 1500 рублей; для должн. лиц - от 2000 до 5000 рублей; для юр. лиц - от 20000 до 100000 рублей; 
-> штраф от 1000 до 1500 рублей; для должн. лиц - от 2000

In [307]:
datapd.to_csv("full.csv")

In [148]:
class Requests:
    
    def __init__ (self, response):
        response = re.split(r'\r\n', response)
        self.query = response[0]
        self.answer = response[2]

In [140]:
f = codecs.open('NotEmptyResponse.txt', encoding = 'windows-1251')
responses = f.read()
f.close()

In [144]:
print (list(responses)[:400])

['ш', 'т', 'р', 'а', 'ф', ' ', 'з', 'а', ' ', 'н', 'е', 'и', 'с', 'п', 'о', 'л', 'н', 'е', 'н', 'и', 'е', ' ', 'т', 'р', 'е', 'б', 'о', 'в', 'а', 'н', 'и', 'я', ',', ' ', 'с', 'о', 'д', 'е', 'р', 'ж', 'а', 'щ', 'е', 'г', 'о', 'с', 'я', ' ', 'в', ' ', 'и', 'с', 'п', 'о', 'л', 'н', 'и', 'т', 'е', 'л', 'ь', 'н', 'о', 'м', ' ', 'л', 'и', 'с', 'т', 'е', '\r', '\n', '"', 'К', 'о', 'д', 'е', 'к', 'с', ' ', 'Р', 'о', 'с', 'с', 'и', 'й', 'с', 'к', 'о', 'й', ' ', 'Ф', 'е', 'д', 'е', 'р', 'а', 'ц', 'и', 'и', ' ', 'о', 'б', ' ', 'а', 'д', 'м', 'и', 'н', 'и', 'с', 'т', 'р', 'а', 'т', 'и', 'в', 'н', 'ы', 'х', ' ', 'п', 'р', 'а', 'в', 'о', 'н', 'а', 'р', 'у', 'ш', 'е', 'н', 'и', 'я', 'х', '"', ' ', 'о', 'т', ' ', '3', '0', '.', '1', '2', '.', '2', '0', '0', '1', ' ', 'N', ' ', '1', '9', '5', '-', 'Ф', 'З', ' ', '(', 'р', 'е', 'д', '.', ' ', 'о', 'т', ' ', '0', '3', '.', '0', '4', '.', '2', '0', '1', '7', ')', '\r', '\n', 'С', 'т', 'а', 'т', 'ь', 'я', ' ', '1', '7', '.', '1', '5', '.', ' ', '$', 'Н', 

In [145]:
responses1 = re.split(r'-----------\r\n', responses)

штраф за неисполнение требования, содержащегося в исполнительном листе
"Кодекс Российской Федерации об административных правонарушениях" от 30.12.2001 N 195-ФЗ (ред. от 03.04.2017)
Статья 17.15. $Неисполнение$ $содержащихся$ в $исполнительном$ $документе$ $требований$ неимущественного характера



In [151]:
responses_list = []
for response in responses1[:-1]:
    responses_list.append(Requests(response))

...лицам, чьи денежные средства и (или) иное $имущество$ привлечены ранее, осуществляются $за$ $счет$ привлеченных денежных средств и (или) иного $имущества$ иных физических лиц и (или) юридических лиц при отсутствии инвестиционной и (или) иной законной $предпринимательской$ или иной деятельности, связанной с использованием привлеченных денежных средств и (или)...


In [162]:
queries_data_file = []
for response in responses_list:
    queries_data_file.append(response.query)
    
queries_data_file = pd.DataFrame(queries_data_file, columns = ['query'])
queries_data_file

,query
0,"штраф за неисполнение требования, содержащегос..."
1,штраф за деятельности без схемы размещения
2,штраф - административное наказание
3,штраф 134
4,штраф за нарушение правил парковки 2012
5,штраф в размере полной стоимости услуг
6,ШТРАФ ЗА УПРАВЛЕНИЕ В НЕТРЕЗВОМ
7,штрафные санкции за непредставление сведений Н...
8,штрафные санкции срок начисления
9,штраф по налогу бухгалтерские налоговые учет


In [204]:
def request_process(responses_list):
    final = []
    for response in responses_list:
        if response.query not in processed_queries:
            goodquery = good_search(response.query)
            txt = re.search(r'Статья (\d+\.\d+\.\d+-\d+\. |\d+\.\d+\.\d+\. |\d+\.\d+\. )', response.answer)
            print (txt)
            if (txt is not None):
                print (response.query)
                txt = txt.group(0)
                num = re.sub(r"Статья ", "", txt)

                num = re.split(r'\.', num)       
                num = num[:-1]
                if (len(num)>2):
                    text_list = datapd[(datapd.chapter == num[0]) & 
                           (datapd.number == num[1]) & 
                           (datapd.subnumber == num[2])]
                else:
                    text_list = datapd[(datapd.chapter == num[0]) & 
                           (datapd.number == num[1]) & 
                           (datapd.subnumber.isnull())]
                num, prec = equality(goodquery, text_list)
                if num == -1:
                    num = None
                    prec = None
                req = {
                    'query': response.query,
                    'id': str(num),
                    'precision': str(prec)
                }
                final.append(req)
                file.write(json.dumps(final[-1]) + '\n')
            else:
                req = {
                    'query': response.query,
                    'id': None,
                    'precision': None
                }
                final.append(req)
                file.write(json.dumps(final[-1]) + '\n')
    file.close()
    return final

In [210]:
query_records = []
tmp_file_name = 'temp1.txt'
if os.path.exists(tmp_file_name):
    with open(tmp_file_name) as file:
        for line in file:
            try:
                query_records.append(json.loads(line))
            except:
                continue
print (len(query_records))
processed_queries = set()
for r in query_records:
    processed_queries.add(r['query'])
file = open(tmp_file_name, 'a')
#-----------------change values here-----|
#---------------------------------------\/
query_records += request_process(responses_list)
df_records = pd.DataFrame(query_records, columns=['query', 'id', 'precision'])
df_full = pd.merge(queries_data_file, df_records, on="query", how="left")

2000
None
<_sre.SRE_Match object; span=(0, 14), match='Статья 12.23. '>
штрафы за нарушения при перевозках
None
<_sre.SRE_Match object; span=(0, 13), match='Статья 7.29. '>
штрафные санкции за нарушение законодательства в сфере закупок товаров, работ, услуг для обеспечения муниципальных нужд
None
<_sre.SRE_Match object; span=(0, 14), match='Статья 19.16. '>
штраф за порчу документов
None
None
None
None
<_sre.SRE_Match object; span=(0, 14), match='Статья 20.15. '>
штрафы за продажу без лицензирования
<_sre.SRE_Match object; span=(0, 13), match='Статья 30.8. '>
штраф выносится на основании решения
None
<_sre.SRE_Match object; span=(0, 13), match='Статья 7.32. '>
штраф подрядчику за изменение условий контракта
None
None
None
None
None
None
<_sre.SRE_Match object; span=(0, 12), match='Статья 7.3. '>
штраф за нарушение условий лицензии
None
None
None
None
None
None
None
<_sre.SRE_Match object; span=(0, 15), match='Статья 5.35.1. '>
штраф можно не платить
None
None
None
None
<_sre.SRE_Match 

In [213]:
df_full

,query,id,precision
0,"штраф за неисполнение требования, содержащегос...",1379,1.0
1,штраф за деятельности без схемы размещения,1028,0.6666666666666666
2,штраф - административное наказание,None,None
3,штраф 134,None,None
4,штраф за нарушение правил парковки 2012,553,0.5
5,штраф в размере полной стоимости услуг,None,None
6,ШТРАФ ЗА УПРАВЛЕНИЕ В НЕТРЕЗВОМ,659,0.5
7,штрафные санкции за непредставление сведений Н...,1523,0.42857142857142855
8,штрафные санкции срок начисления,None,None
9,штраф по налогу бухгалтерские налоговые учет,None,None


In [290]:
tst = df_without_none[(df_without_none['precision'] > '0.67') & (df_without_none['precision'] != 'None')]

In [291]:
tst

,query,id,precision
0,"штраф за неисполнение требования, содержащегос...",1379,1.0
23,штраф за непредставление сведений для осуществ...,1122,1.0
42,штрафы по контракту закупки,250,1.0
45,штраф за сведения пфр,1272,0.75
48,штрафы пени по налогам взносам,1126,1.3333333333333333
51,штрафы за вред федеральным дорогам,729,1.0
52,штраф за передачу исполнительного лица,1556,1.0
57,штрафы за нарушение законодательства об инвалидах,75,1.0
63,штрафные санкции омс,1270,0.75
72,штраф за нарушение работы с денежной наличностью,1115,1.0


In [292]:
tst.to_csv('goodrequests.csv')

In [293]:
tst = pd.read_csv('goodrequests.csv')

In [294]:
tst

,Unnamed: 0,query,id,precision
0,0,"штраф за неисполнение требования, содержащегос...",1379,1.000000
1,23,штраф за непредставление сведений для осуществ...,1122,1.000000
2,42,штрафы по контракту закупки,250,1.000000
3,45,штраф за сведения пфр,1272,0.750000
4,48,штрафы пени по налогам взносам,1126,1.333333
5,51,штрафы за вред федеральным дорогам,729,1.000000
6,52,штраф за передачу исполнительного лица,1556,1.000000
7,57,штрафы за нарушение законодательства об инвалидах,75,1.000000
8,63,штрафные санкции омс,1270,0.750000
9,72,штраф за нарушение работы с денежной наличностью,1115,1.000000


In [295]:
tst = tst.drop('Unnamed: 0', axis = 1)

In [308]:
outversion = tst
outversion['text'] = None
outversion['answer'] = None

for num, row in enumerate(outversion['id']):
    print (num)
    outversion.loc[num, 'text'] = datapd.iloc[row]['text_out']
    outversion.loc[num, 'answer'] = datapd.iloc[row]['answer_out']

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [248]:
datapd

,chapter,number,subnumber,title,part,text,answer,answer_out,text_out
0,5,1,None,Нарушение права гражданина на ознакомление со ...,0,Нарушение права гражданина на ознакомление со ...,наложение административного штрафа в размере ...,-> от 1000 до 1500 рублей;,КоАП ст. 5.1. Нарушение права гражданина на...
1,5,2,None,Утратила силу. - Федеральный закон от 04.07.20...,0,None,None,None,КоАП ст. 5.2. Утратила силу...
2,5,3,None,"Неисполнение решения избирательной комиссии, к...",1.,"Неисполнение решения избирательной комиссии, к...",наложение административного штрафа на должнос...,-> для должн. лиц - от 1000 до 2000 рублей; дл...,КоАП ст. 5.3. ч.1. Неисполнение решения избира...
3,5,3,None,"Неисполнение решения избирательной комиссии, к...",2.,"Непредставление государственными органами, орг...",наложение административного штрафа на должнос...,-> для должн. лиц - от 1000 до 1500 рублей; дл...,КоАП ст. 5.3. ч.2. Непредставление государстве...
4,5,4,None,Нарушение порядка представления сведений об из...,0,Нарушение установленного законом порядка предс...,наложение административного штрафа на должнос...,-> для должн. лиц - от 1000 до 5000 рублей;,КоАП ст. 5.4. Нарушение порядка представления ...
5,5,5,None,Нарушение порядка участия средств массовой инф...,1.,"Нарушение главным редактором, редакцией средст...",наложение административного штрафа на граждан...,-> от 500 до 2500 рублей; для юр. лиц - от 300...,КоАП ст. 5.5. ч.1. Нарушение главным редакторо...
6,5,5,None,Нарушение порядка участия средств массовой инф...,2.,Непредоставление государственной или муниципал...,наложение административного штрафа на должнос...,-> для должн. лиц - от 3000 до 4000 рублей; дл...,КоАП ст. 5.5. ч.2. Непредоставление государств...
7,5,6,None,"Нарушение прав члена избирательной комиссии, к...",1.,"Нарушение прав члена избирательной комиссии, к...",наложение административного штрафа на граждан...,-> от 500 до 1000 рублей; для должн. лиц - от ...,КоАП ст. 5.6. ч.1. Нарушение прав члена избира...
8,5,6,None,"Нарушение прав члена избирательной комиссии, к...",2.,"Выдача председателем, заместителем председател...",наложение административного штрафа в размере ...,-> от 1500 до 2000 рублей;,КоАП ст. 5.6. ч.2. Выдача председателем...
9,5,7,None,Отказ в предоставлении отпуска для участия в в...,0,Отказ работодателя предоставить предусмотренны...,наложение административного штрафа в размере ...,-> от 1500 до 2000 рублей;,КоАП ст. 5.7. Отказ в предоставлении отпуска...


In [309]:
outversion = outversion.drop('id', 1)

In [312]:
outversion = outversion.dropna()

In [313]:
outversion.to_csv('project.csv')

In [314]:
outversion.to_excel('project1.xlsx')